In [223]:
from pyproj import Proj, transform
import stateplane
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely import geometry
import geojson

In [224]:
data=pd.read_csv('2016_Racial_Profiling_Dataset_Arrests.csv')
keys=data.keys()

In [225]:
with open('viz/data/austin_area_zip_codes.geojson') as file:
    data2 = geojson.load(file)
data2 = data2['features']
zippolygondict = {}
for n in range(len(data2)):
    zippolygondict[int(data2[n]['properties']['zipcode'])] = geometry.Polygon([[p[1], p[0]] for p in data2[n]['geometry']['coordinates'][0][0]])

In [226]:
from shapely.geometry import Point
keys = list(zippolygondict.keys())
def checkzipcode(X,Y):
    #X and Y are coordinates as float objects
    for m in range(len(keys)):
        if zippolygondict[keys[m]].contains(Point(X,Y)):
            return keys[m]

In [227]:
X = data['X_COORDINATE'].as_matrix().astype(float)
Y = data['Y_COORDINATE'].as_matrix().astype(float)
zipcodes = np.zeros((len(data['SEX']),))
for n in range(len(X)):
    coordinates = stateplane.to_latlon(X[n]*.3048, Y[n]*.3048, epsg=None, fips='4203', abbr=None)
    X[n] = coordinates[0]
    Y[n] = coordinates[1]
    zipcodes[n] = checkzipcode(X[n],Y[n])
data['X_COORDINATE'] = X
data['Y_COORDINATE'] = Y
data['ZIP_CODE'] = zipcodes
data.to_csv('2016_Racial_Profiling_Dataset_Arrests_LonLat.csv')

In [11]:
np.sum(data['SEX'] == 'M')/len(data['SEX'])

0.74989111498257843

In [20]:
np.sum(data['APD_RACE_DESC'] == 'BLACK')/len(data['SEX'])

0.23432055749128919